### Pickle verification results

In [ ]:
import json
import data
import evaluation

In [1]:
import data
data = data.Data("config.json", "nagini_examples/incremental")

In [ ]:
RESULT_FILE = "nagini_examples/incremental/verif_result.json"
data = data.Data("config.json", "nagini_examples/incremental")
ev = evaluation.Evaluation(data)
examples = ev.data.get_list_of_examples("list")
# examples = ["prepend"]
verif_result:dict[str,str] = {}

# with open(RESULT_FILE, "rb") as f:
#     verif_result = pickle.load(f)
# print(verif_result)

for example in examples:
    program_snippet = ev.data.get_example("list", example, "unverified")
    result = ev.verify_program_snippet("list", program_snippet)
    verif_result[example] = str(result)

with open(RESULT_FILE, "w") as f:
    json.dump(verif_result, f, indent=4)

In [ ]:
verif_result

In [ ]:
import pickle
with open("nagini_examples/verif_result_sorting.pkl", "rb") as f:
    verif_result = pickle.load(f)
verif_result


In [ ]:
import json
with open("nagini_examples/verif_result_sorting.json", "w") as f:
    json.dump(verif_result, f, indent=4)